New TxMM notebook

In [2]:
! pip install transformers datasets accelerate nvidia-ml-py3
# ! pip install torch --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu116


     ------------------------------------- 191.5/191.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19181 sha256=2fc70dc1861eda2684f8c1385ee3c67cbc17441167f9bbe8e42ba456414c8657
  Stored in directory: C:\Users\roell\AppData\Local\Temp\pip-ephem-wheel-cache-j7qc82ak\wheels\19\3d\20\9bf7f9bd95806ed38fdc85430b24f3fe52a68556f94104d6b6
Successfully built nvidia-ml-py3


In [3]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [9]:
import torch

# CUDA 11.6 needed!
torch.cuda.is_available()

torch.ones((1, 1)).to("cuda")


tensor([[1.]], device='cuda:0')

In [11]:
print_gpu_utilization()

GPU memory occupied: 647 MB.
